## A1

#### Daten einlesen

Es empfiehlt sich, den gesamten Text in einen String einzulesen, dann den Zeilenvorschub zu entfernen und alles in Kleinbuchstaben zu verwandeln.

In [ ]:
f = open('eingabe.txt',"r",encoding='utf-8')
text = f.read()                         # die komplette Eingabe in einen String
text = text.replace('\n','').lower()

#### Reguläre Ausdrücke
Mit regulären Ausdrücken können wir Texte nach Mustern durchsuchen

```
\w steht für einen beliebigen Buchstaben
\w+ steht für ein oder mehrere beliebe Buchstaben, also ein Wort
\w+ \w+   steht für 2 Worte mit einem Leerzeichen dazwischen
```

In [4]:
import re
# es sollen die Wortfolgen 'ein _ _ Satz' gefunden werden. _ steht für ein beliebiges Wort.
regex = 'ein \w+ \w+ Satz'
text = 'Dies ist ein wirklich kurzer Satz. Morgen ist Freitag. Auch dies könnte ein sehr schöner Satz werden.'
a = re.findall(regex,text)
print(a)
 

['ein wirklich kurzer Satz', 'ein sehr schöner Satz']
